In [1]:
from __future__ import print_function

## Issues
This notebook is a copy of the script `ClassifcationKeras.py`. The script does not run whereas this notebook does.

In [2]:
import sys
print(sys.path)
print(sys.version)
print(sys.version_info)
print(sys.executable)

['', '/mnt/share/cern_CentOS7/root/v6.12.06/lib', '/home/mikew/git_repos/ml4np/pyroot', '/cern/cernlib/x86_64_rhel6_4.7.2/2005/lib', '/home/mikew/.conda/envs/keras/lib/python27.zip', '/home/mikew/.conda/envs/keras/lib/python2.7', '/home/mikew/.conda/envs/keras/lib/python2.7/plat-linux2', '/home/mikew/.conda/envs/keras/lib/python2.7/lib-tk', '/home/mikew/.conda/envs/keras/lib/python2.7/lib-old', '/home/mikew/.conda/envs/keras/lib/python2.7/lib-dynload', '/home/mikew/.local/lib/python2.7/site-packages', '/home/mikew/.conda/envs/keras/lib/python2.7/site-packages', '/home/mikew/.conda/envs/keras/lib/python2.7/site-packages/Mako-1.0.7-py2.7.egg', '/home/mikew/.conda/envs/keras/lib/python2.7/site-packages/IPython/extensions', '/home/mikew/.ipython']
2.7.15 |Anaconda, Inc.| (default, May  1 2018, 23:32:55) 
[GCC 7.2.0]
sys.version_info(major=2, minor=7, micro=15, releaselevel='final', serial=0)
/home/mikew/.conda/envs/keras/bin/python


### Comparison with terminal
```
['/home/mikew/git_repos/ml4np/pyroot', '/mnt/share/cern_CentOS7/root/v6.12.06/lib', '/home/mikew/git_repos/ml4np/pyroot', '/cern/cernlib/x86_64_rhel6_4.7.2/2005/lib', '/home/mikew/.conda/envs/keras/lib/python27.zip', '/home/mikew/.conda/envs/keras/lib/python2.7', '/home/mikew/.conda/envs/keras/lib/python2.7/plat-linux2', '/home/mikew/.conda/envs/keras/lib/python2.7/lib-tk', '/home/mikew/.conda/envs/keras/lib/python2.7/lib-old', '/home/mikew/.conda/envs/keras/lib/python2.7/lib-dynload', '/home/mikew/.local/lib/python2.7/site-packages', '/home/mikew/.conda/envs/keras/lib/python2.7/site-packages', '/home/mikew/.conda/envs/keras/lib/python2.7/site-packages/Mako-1.0.7-py2.7.egg']
```

In [3]:
from ROOT import TMVA, TFile, TTree, TCut
from subprocess import call
from os.path import isfile, isdir
import os

Welcome to JupyROOT 6.12/06


In [4]:
from keras.models import Sequential
from keras.layers.core import Dense, Activation, Dropout
from keras.regularizers import l2
from keras import initializers
from keras.optimizers import SGD, Nadam

Using Theano backend.


In [5]:
# Setup TMVA
TMVA.Tools.Instance()
TMVA.PyMethodBase.PyInitialize()

In [6]:
# sav file in given location
i = 0
model_path = './models/'

while isfile('{0}model{1}.h5'.format(model_path, i)):
    i += 1
name = 'PyKeras-{0}'.format(i)

In [7]:
output = TFile.Open('TMVA.root', 'UPDATE')
factory = TMVA.Factory('TMVAClassification', output,
        '!V:!Silent:Color:DrawProgressBar:Transformations=D,G:AnalysisType=Classification')

fnameSignal = '../data/rho10_tm1_sm1_stat/Signal.root'
fnameBackground = '../data/rho10_tm1_sm1_stat/Background.root'

# Load data
if not isfile(fnameSignal):
    print('File not found: {0}'.format(fnameSignal))
if not isfile(fnameBackground):
    print('File not found: {0}'.format(fnameBackground))

dataSignal = TFile.Open(fnameSignal)
dataBackground = TFile.Open(fnameBackground)
signal = dataSignal.Get('HSParticles')
background = dataBackground.Get('HSParticles')

In [8]:
# make data loader
dataloader = TMVA.DataLoader('dataset')

# list of variables to include
include = ['ElTime', 'ElP', 'ElTh', 'ElPhi', 'PTime', 'PP', 'PTh', 'PPhi', 'PimTime', 'PimP', 'PimTh', 'PimPhi', 'PipTime', 'PipP', 'PipTh', 'PipPhi']
Nvar = len(include)
# loop over branches
for branch in signal.GetListOfBranches():
    if branch.GetName() in include:
        dataloader.AddVariable(branch.GetName())

dataloader.AddSignalTree(signal, 1.0)
dataloader.AddBackgroundTree(background, 1.0)
dataloader.PrepareTrainingAndTestTree(TCut(''),
        'nTrain_Signal=4000:nTrain_Background=4000:SplitMode=Random:NormMode=NumEvents:!V')

DataSetInfo              : [dataset] : Added class "Signal"
                         : Add Tree HSParticles of type Signal with 5616 events
DataSetInfo              : [dataset] : Added class "Background"
                         : Add Tree HSParticles of type Background with 140208 events
                         : Dataset[dataset] : Class index : 0  name : Signal
                         : Dataset[dataset] : Class index : 1  name : Background


In [9]:
model = Sequential()
model.add(Dense(64, activation='relu', kernel_regularizer=l2(1e-5), input_dim=Nvar))
model.add(Dense(32, activation='relu', kernel_regularizer=l2(1e-5), input_dim=Nvar))
#model.add(Dropout(0.))
model.add(Dense(16, activation='relu', kernel_regularizer=l2(1e-5)))
model.add(Dropout(0.1))
#model.add(Dense(32,  activation='relu', kernel_regularizer=l2(1e-5)))
model.add(Dense(2, activation='softmax'))

# Set loss and optimizer
model.compile(loss='categorical_crossentropy', optimizer=Nadam(), metrics=['accuracy',])

# Store model to file
model.save('{0}model{1}.h5'.format(model_path, i))
model.save('model.h5')
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 64)                1088      
_________________________________________________________________
dense_2 (Dense)              (None, 32)                2080      
_________________________________________________________________
dense_3 (Dense)              (None, 16)                528       
_________________________________________________________________
dropout_1 (Dropout)          (None, 16)                0         
_________________________________________________________________
dense_4 (Dense)              (None, 2)                 34        
Total params: 3,730
Trainable params: 3,730
Non-trainable params: 0
_________________________________________________________________


In [10]:
# book methods

factory.BookMethod(dataloader, TMVA.Types.kPyKeras, name,        'H:!V:VarTransform=D,G:FilenameModel=model.h5:NumEpochs=40:BatchSize=32')

# Run training, test and evaluation
factory.TrainAllMethods()
factory.TestAllMethods()
factory.EvaluateAllMethods()

/home/mikew/.conda/envs/keras/lib/python2.7/site-packages/keras/models.py:981: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Train on 8000 samples, validate on 137824 samples
Epoch 1/40
8000/8000 [==============================] - 1s 156us/step - loss: 0.3078 - acc: 0.8679 - val_loss: 0.3038 - val_acc: 0.8385

Epoch 00001: val_loss improved from inf to 0.30376, saving model to dataset/weights/TrainedModel_PyKeras-1.h5
Epoch 2/40
8000/8000 [==============================] - 1s 149us/step - loss: 0.2096 - acc: 0.9156 - val_loss: 0.2891 - val_acc: 0.8497

Epoch 00002: val_loss improved from 0.30376 to 0.28914, saving model to dataset/weights/TrainedModel_PyKeras-1.h5
Epoch 3/40
8000/8000 [==============================] - 1s 151us/step - loss: 0.1896 - acc: 0.9251 - val_loss: 0.2775 - val_acc: 0.8672

Epoch 00003: val_loss improved from 0.28914 to 0.27748, saving model to dataset/weights/TrainedModel_PyKeras-1.h5
Epoch 4/40
8000/8000 [==============================] - 1s 150us/step - loss: 0.1751 - acc: 0.9306 - val_loss: 0.1835 - val_acc: 0.9124

Epoch 00004: val_loss improved from 0.27748 to 0.18354, saving m